<a href="https://colab.research.google.com/github/Jojocko/Feedback-users-supply-chain/blob/main/M%C3%A9thode_1_clustering_par_TFidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Datasets NLP/dataset_supplychain.csv')
df.head(5)

,Commentaire,star,date,client,reponse,source,company,langage,cleaned_words,cleaned_lemma,Sentiment
0,"Colis commandé mardi 28 fevrier 23 , livraiso...",1,2023-03-07,Toto,NaN,TrustPilot,Amazon,fr,colis commandé mardi fevrier livraison jeu...,colis commander mardi fevrier livraison je...,__label__NEGATIVE
1,Amazon avec sa politique de retour est la pire...,1,2023-03-07,nasri eddine,NaN,TrustPilot,Amazon,fr,amazon politique pire contrairement ...,amazon politique pire contrairement ...,__label__NEGATIVE
2,Dieu sait que j'en connais des déboires avec l...,4,2023-03-07,Amandine,NaN,TrustPilot,Amazon,fr,dieu connais déboires sites marchands ...,dieu connaître déboire site marchand a...,__label__POSITIVE
3,"Nul, preleve une commande que je n'ai jamais r...",1,2023-03-07,Bob Brico,NaN,TrustPilot,Amazon,fr,preleve commande jamais reçu service cl...,prelev commande jamais recevoir service...,__label__NEGATIVE
4,Colis soit disant livré mais jamais reçu donc ...,1,2023-03-06,Client,NaN,TrustPilot,Amazon,fr,colis disant livré jamais reçu perdu non re...,colis dire livrer jamais recevoir perdre no...,__label__NEGATIVE


In [10]:
# Pré-traîtement des données

# Traitement du df

df = df.drop(['client', 'langage', 'reponse'], axis=1)
df.rename(columns={'Sentiment': 'sentiment', 'Commentaire': 'commentaire'}, inplace=True)
df['sentiment'] = df['sentiment'].replace({'__label__POSITIVE': 'positif', '__label__NEGATIVE': 'negatif', '__label__NEUTRAL': 'neutre'})
df['date'] = df['date'].fillna(method="ffill")
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# Traitement des commentaires déjà lemmatisés

import re
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('french'))
company = {'Fnac', 'Fnac', 'Amazon', 'amazon', 'CDiscount', 'cdiscount'}
stop_words.update(company)
stop_words.update(['service', 'client'])

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def contractions(text):
    text = text.replace("c'est", "ce est").replace("j'ai", "je ai")
    return text

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = contractions(w)
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ?.!,¿]+", " ", w)
    w = re.sub(r'\b[a-zA-Z]{1,2}\b', '', w)

    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

df['cleaned_lemma'] = df['cleaned_lemma'].apply(preprocess_sentence)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
# Clustering sur commentaires négatifs avec tfidf

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

negative_comments = df[df['sentiment'] == 'negatif']

stop_words_list = list(stop_words)

tfidf_vectorizer = TfidfVectorizer(max_df=0.7, ngram_range=(2, 3), stop_words=stop_words_list)
vec_reponses_neg = tfidf_vectorizer.fit_transform(negative_comments.cleaned_lemma)

km_neg = KMeans(n_clusters=8, random_state=42)
km_neg.fit(vec_reponses_neg)

negative_comments['cluster'] = km_neg.labels_

tfidf_per_cluster_neg = {}

for cluster_num in range(8):
    documents_in_cluster = negative_comments[negative_comments['cluster'] == cluster_num].cleaned_lemma

    X_tfidf = tfidf_vectorizer.transform(documents_in_cluster)

    scores = X_tfidf.toarray().mean(axis=0)
    terms = tfidf_vectorizer.get_feature_names_out()

    tfidf_per_cluster_neg[cluster_num] = pd.DataFrame(data=scores, index=terms, columns=["TF-IDF"]).sort_values(by="TF-IDF", ascending=False)

    print(f"Top terms for cluster {cluster_num}:")
    print(tfidf_per_cluster_neg[cluster_num].head(10))
    print("\n")


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fnac'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-11-af1a8798fe67>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_comments['cluster'] = km_neg.labels_


Top terms for cluster 0:
                          TF-IDF
point relais            0.022675
date livraison          0.019565
point relai             0.013553
deconseiller fortement  0.012890
colis priver            0.010607
attendre jour           0.008237
livraison point         0.007350
livrer point            0.007052
devoir attendre         0.005143
livraison point relais  0.004442


Top terms for cluster 1:
                         TF-IDF
recevoir colis         0.039579
recevoir article       0.016244
article recevoir       0.012960
renvoyer article       0.010382
jamais recevoir colis  0.007770
jamais recevoir        0.007277
boite lettre           0.004325
commande recevoir      0.003737
recevoir mail          0.003631
aujourd hui            0.003605


Top terms for cluster 2:
                    TF-IDF
passer commande   0.002704
bon achat         0.002292
annuler commande  0.001953
commande passer   0.001668
raccrocher nez    0.001576
frais port        0.001567
delai livraison  

Nous avons décidé de clusteriser sur les scores TF-IDF afin d'identifier des termes significatifs moins fréquents mais tout autant informatifs.  

In [12]:
# Calcul du score de silhouette

from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(vec_reponses_neg, km_neg.labels_)
print(f"Score de silhouette: {silhouette_avg}")

Score de silhouette: 0.0005164383630886183


Le score de silhouette très bas indique que le modèle a du mal à distinguer clairement entre différents types de commentaires négatifs. Cette difficulté est aussi démontrée par la prédominance de thèmes similaires relatifs aux problèmes de commande dans 5 des 7 clusters, malgré plusieurs ajustements des paramètres, y compris le nombre de clusters et les seuils de fréquence des termes.

Pour surmonter ces limitations, nous envisageons d'adopter une approche de multi-clustering pour notre deuxième tentative, avec l'objectif de segmenter plus finement les commentaires.